In [1]:
import bokeh.plotting as bkp
import numpy as np

import arviz as az
import pystan

In [2]:
SEED = 23432532

In [3]:
az.backends.output_notebook()

Loading BokehJS ...

# Simple linear regression problem 

$y = m x + b$

where

$res = y - (m x + b)$

and 

$res \sim normal(0, e)$

In [4]:
prior_code = """
data {
    int<lower=0> N;
    vector[N] x;
}
parameters {
    real m;
    real b;
    real<lower=0> e;
}
model {
    m ~ student_t(3,0,1);
    b ~ student_t(3,0,1);
    e ~ student_t(3,0,1);   
}
generated quantities {
    vector[N] y;
    vector[N] y_hat;
    for (n in 1:N) {
        y[n] = m * x[n] + b;
        y_hat[n] = normal_rng(m * x[n] + b, e);
    }
}
"""

In [5]:
%time stan_prior = pystan.StanModel(model_code=prior_code, extra_compile_args=["-flto"])

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_094a6f81f169e8d1984dba03ff5cffba NOW.


Wall time: 32.5 s


In [6]:
np.random.seed(SEED + 1)
N_prior = 30
x_prior = np.linspace(0, 100, N_prior) + np.random.randn(N_prior) / 100

In [7]:
stan_prior_data = dict(
    N = N_prior,
    x = x_prior,
)

In [8]:
%time stan_prior_fit = stan_prior.sampling(data=stan_prior_data, check_hmc_diagnostics=False)

Wall time: 1.92 s


# Prior

In [9]:
az.plot_trace(stan_prior_fit, var_names=['m', 'b', 'e'], backend="bokeh")

array([[Figure(id='1002', ...), Figure(id='1042', ...)],
       [Figure(id='1082', ...), Figure(id='1122', ...)],
       [Figure(id='1161', ...), Figure(id='1201', ...)]], dtype=object)

In [10]:
y_prior = stan_prior_fit.extract(pars=['y'])['y']
y_prior_hat = stan_prior_fit.extract(pars=['y_hat'])['y_hat']

In [11]:
y_prior.shape

(4000, 30)

# Prior predictive

In [12]:
p = bkp.figure(height=300, width=500)
for i, yph in enumerate(y_prior_hat[:200]):
    p.circle(x_prior, yph, color='blue', alpha=0.1, size=4)
bkp.show(p)

In [13]:
p = bkp.figure(height=300, width=500)
for i, yph in enumerate(y_prior_hat[:200]):
    p.line(x_prior, yph, color='blue', alpha=0.1)
bkp.show(p)

In [14]:
np.random.seed(SEED + 2)
N = 20

# linear model parameters
m = 0.4
b = 34
e = 5 # noise term

# x, y_true and observed y
x = np.sort(np.random.rand(N)*100)
y_true = x * m + b
e_arr = np.random.randn(N) * e
y = y_true + e_arr

In [15]:
p = bkp.figure(height=300, width=500)

p.circle(x, y, color='blue', size=6)
p.line(x, y_true, color="black")

bkp.show(p)

In [16]:
model_code = """
data {
    int<lower=0> N;
    vector[N] y;
    vector[N] x;
}
parameters {
    real m;
    real b;
    real<lower=0> e;
}
model {
    m ~ student_t(3,0,1);
    b ~ student_t(3,0,1);
    e ~ student_t(3,0,1);
    y ~ normal(m * x + b, e);
}
generated quantities {
    // posterior predictive distribution
    vector[N] log_likelihood;
    vector[N] y_hat;
    for (n in 1:N) {
        log_likelihood[n] = normal_lpdf(y[n] | m*x[n]+b,e);
        y_hat[n] = normal_rng(m*x[n]+b,e);
    }
}
"""

In [17]:
%time stan_model = pystan.StanModel(model_code=model_code, extra_compile_args=["-flto"])

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4969c9a075197a0e56a14e2cc29962a9 NOW.


Wall time: 33.8 s


In [18]:
stan_data = dict(
    N = N,
    y = y,
    x = x,
)

In [19]:
stan_map = stan_model.optimizing(data=stan_data, seed=12354)
stan_map

OrderedDict([('m', array(0.89258782)),
             ('b', array(0.52902266)),
             ('e', array(19.72441728)),
             ('log_likelihood',
              array([-5.86756131, -4.9889332 , -5.10545787, -4.75316625, -5.43017053,
                     -5.91018772, -5.02846349, -4.78728118, -4.09266246, -4.09266036,
                     -3.91686394, -3.93579576, -4.02492406, -4.04497756, -3.96468856,
                     -3.90106315, -3.97830182, -4.09459791, -4.14670169, -3.93654794])),
             ('y_hat',
              array([-32.24297386, -26.74856504,  -8.74104698,  50.38402241,
                      36.7226536 ,   8.64432455,  34.93593466,  20.24094864,
                      45.40065402,  45.22424705,  58.65821452,  49.37529496,
                      72.498357  ,  55.26243862,  59.31628236,  88.47760114,
                      90.88808749,  62.015193  ,  73.83793205,  75.08444936]))])

In [20]:
stan_fit = stan_model.sampling(data=stan_data, seed=12355)

In [21]:
idata = az.from_pystan(posterior=stan_fit, 
                       posterior_predictive="y_hat", 
                       prior=stan_prior_fit, 
                       prior_predictive="y_hat", 
                       observed_data="y", 
                       constant_data=["x", "N"], 
                       log_likelihood="log_likelihood")

In [22]:
az.summary(idata, var_names=['m', 'b', 'e'])

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
m,0.397,0.039,0.324,0.472,0.001,0.001,1747.0,1726.0,1757.0,1663.0,1.0
b,35.350,2.198,31.191,39.502,0.052,0.037,1762.0,1762.0,1782.0,1660.0,1.0
e,5.780,0.929,4.203,7.551,0.021,0.015,2017.0,1929.0,2166.0,2066.0,1.0


# Posterior

In [23]:
import xarray as xr
xr.set_options(display_style = "html")

In [24]:
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 1000)
Coordinates:
  * chain    (chain) int32 0 1 2 3
  * draw     (draw) int32 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    m        (chain, draw) float64 0.3819 0.3721 0.3557 ... 0.413 0.3675 0.4229
    b        (chain, draw) float64 35.74 37.38 36.65 36.43 ... 35.35 38.15 32.58
    e        (chain, draw) float64 5.631 7.128 6.496 5.001 ... 3.789 5.562 6.555
Attributes:
    created_at:                 2020-01-14T09:30:16.638186
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [25]:
az.plot_trace(idata, backend="bokeh")

array([[Figure(id='11460', ...), Figure(id='11500', ...)],
       [Figure(id='11540', ...), Figure(id='11580', ...)],
       [Figure(id='11619', ...), Figure(id='11659', ...)]], dtype=object)

In [26]:
from bokeh.layouts import gridplot

In [27]:
axes = az.plot_density(idata, var_names=['m', 'b', 'e'], backend="bokeh", shade=0.1, show=False)
for ax in axes.ravel():
    param = locals().get(ax.title.text)
    ax.circle(param, 0, color="red", size=20, alpha=0.2)
bkp.show(gridplot(axes.tolist()))

In [28]:
az.plot_pair(idata, var_names=['m', 'b', 'e'], backend="bokeh", figsize=(10,10), divergences=True)

array([[Figure(id='15169', ...), None],
       [Figure(id='15209', ...), Figure(id='15249', ...)]], dtype=object)

# Posterior predictive

In [29]:
idata.posterior_predictive

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 1000, y_hat_dim_0: 20)
Coordinates:
  * chain        (chain) int32 0 1 2 3
  * draw         (draw) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * y_hat_dim_0  (y_hat_dim_0) int32 0 1 2 3 4 5 6 7 ... 12 13 14 15 16 17 18 19
Data variables:
    y_hat        (chain, draw, y_hat_dim_0) float64 39.43 41.84 ... 74.0 66.83
Attributes:
    created_at:                 2020-01-14T09:30:16.651262
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [30]:
idata.posterior_predictive.y_hat.shape

(4, 1000, 20)

In [31]:
idata.posterior_predictive.y_hat[np.random.randint(0,3), np.random.randint(0,999)].values

array([43.2453609 , 31.11582377, 37.81885392, 42.26356629, 43.66884144,
       37.53218018, 49.27976104, 43.8931509 , 45.31345353, 50.70661231,
       63.58232877, 57.4630035 , 53.89491406, 56.79731362, 70.88462912,
       57.04593655, 67.86768598, 58.62210184, 75.48041943, 70.81871129])

In [32]:
p = bkp.figure(height=300, width=500)
yh = idata.posterior_predictive.y_hat
for i in range(200):
    p.circle(x, yh[np.random.randint(0,3), np.random.randint(0,999)].values, color='orange', size=4, alpha=0.1)
p.circle(x, y, color='black', size=10)
bkp.show(p)

# Posterior

In [33]:
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 1000)
Coordinates:
  * chain    (chain) int32 0 1 2 3
  * draw     (draw) int32 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    m        (chain, draw) float64 0.3819 0.3721 0.3557 ... 0.413 0.3675 0.4229
    b        (chain, draw) float64 35.74 37.38 36.65 36.43 ... 35.35 38.15 32.58
    e        (chain, draw) float64 5.631 7.128 6.496 5.001 ... 3.789 5.562 6.555
Attributes:
    created_at:                 2020-01-14T09:30:16.638186
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [34]:
stacked_idata = idata.posterior.stack(samples=["chain", "draw"])
m_hat = stacked_idata['m'].values
b_hat = stacked_idata['b'].values

In [35]:
p = bkp.figure(height=300, width=500)
slices = []
for i in range(200):
    random_slice = np.random.randint(m_hat.shape[0])
    p.line(x, x*m_hat[random_slice]+b_hat[random_slice], color='orange', alpha=0.07)
    if i > 200:
        break
p.circle(x, y, color='blue',)
p.line(x, y_true, color='black')
bkp.show(p)

In [36]:
loo_info = az.loo(idata, pointwise=True, scale="log")
loo_info

Computed from 4000 by 20 log-likelihood matrix

         Estimate       SE
elpd_loo   -66.16     3.03
p_loo        2.84        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)       20  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

In [37]:
az.plot_khat(loo_info.pareto_k, backend="bokeh", figsize=(10,10), show_bins=True)

Figure(id='26986', ...)

In [38]:
loo_info.pareto_k

<xarray.DataArray 'pareto_shape' (log_likelihood_dim_0: 20)>
array([ 0.15624422,  0.19526684,  0.21552892,  0.31892522,  0.16231946,
        0.36405568,  0.0425763 ,  0.0541253 ,  0.04955603,  0.04130483,
       -0.03137004,  0.30196391,  0.11573286,  0.256313  ,  0.30166927,
        0.01610363,  0.13845106,  0.25003765,  0.28798406,  0.26315951])
Coordinates:
  * log_likelihood_dim_0  (log_likelihood_dim_0) int32 0 1 2 3 4 ... 16 17 18 19

In [39]:
%load_ext version_information

In [40]:
from datetime import datetime

In [41]:
print(datetime.now())

2020-01-14 11:30:21.109791


In [42]:
%version_information numpy, cython, pystan, arviz, bokeh, xarray

Software versions
Python 3.7.5 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.9.0
OS Windows 10 10.0.17763 SP0
numpy 1.17.5
cython 0.29.13
pystan 2.19.1.1
arviz 0.6.1
bokeh 1.4.0
xarray 0.14.1
Tue Jan 14 11:30:21 2020 FLE Standard Time